In [1]:



data = {
    "board": {
        "width": 8,
        "height": 7,
        "start":[[1, 0, 1, 1, 2, 2, 1,1],
                [2, 3, 1, 1, 0, 0, 2,1],
                [3, 0, 2, 1, 1, 1, 1,1],
                [3, 0, 0, 2, 2, 3, 1,1],
                [2, 2, 3, 2, 0, 2, 2,1],
                [3, 3, 1, 0, 3, 2, 3,1],
                [0, 1, 0, 0, 0, 2, 2,1]],
        "goal": 
                [[1, 0, 1, 1, 2, 2, 1,1],
                [2, 3, 1, 1, 0, 0, 2,1],
                [3, 0, 1,  1, 1,2,1,1],
                [3, 0, 2, 1, 0, 2,3,1],
                [2, 2, 0, 2, 2, 1, 2, 3],
                [3, 3, 1, 0, 3, 3,1,2],
                [0, 1, 0, 0,  2, 1,2,0]],
    },
    "general": {
        "n": 1,
        "patterns": [
            {
                "p": 25,
                "width": 3,
                "height": 3,
                "cells": [
                    [0,1,0],
                    [1,0,1],
                    [1,1,0]
                ]
            }
        ]
    }
}


def print_board(board):
    for row in board:
        print(" ".join(row))

def generate_board_matrix(data):
    board_data = data["board"]
    width = board_data["width"]
    height = board_data["height"]
    start = board_data["start"]
    goal = board_data["goal"]
    
    
    return start, goal


def generate_fixed_die_matrix(matrix_id):
    sizes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    type_count = 3
    if matrix_id < 0 or matrix_id >= len(sizes) * type_count - 2:
        raise ValueError("Invalid matrix ID")
    if matrix_id == 0:
        return [[1]]
    adjusted_id = matrix_id + 2 if matrix_id >= 1 else matrix_id

    size_index = adjusted_id // type_count
    type_index = adjusted_id % type_count

    n = sizes[size_index]

    if type_index == 0:
        matrix = [[1 for _ in range(n)] for _ in range(n)]
    elif type_index == 1:
        matrix = [[1 if i % 2 == 0 else 0 for _ in range(n)] for i in range(n)]
    elif type_index == 2:
        matrix = [[1 if j % 2 == 0 else 0 for j in range(n)] for _ in range(n)]

    return matrix

def generate_pattern_matrix(data, matrix_id):
    patterns = data["general"]["patterns"]
    for pattern in patterns:
        if pattern["p"] == matrix_id:
            return [list(map(int, row)) for row in pattern["cells"]]
    raise ValueError("Pattern ID not found in data")

def generate_matrix(matrix_id, data):
    if matrix_id < 25:
        return generate_fixed_die_matrix(matrix_id)
    else:
        return generate_pattern_matrix(data, matrix_id)
        

In [2]:
def apply_die_cutting(board, pattern, x, y, direction):
    height = len(board)
    width = len(board[0])
    p_height = len(pattern)
    p_width = len(pattern[0])

    lifted_elements = []
    for i in range(p_height):
        for j in range(p_width):
            if pattern[i][j] == 1:
                if 0 <= y + i < height and 0 <= x + j < width:
                    lifted_elements.append(board[y + i][x + j])
                    board[y + i][x + j] = None

    if direction == 1: 
        for j in range(width):
            new_col = [board[i][j] for i in range(height) if board[i][j] is not None]
            new_col = [None] * (height - len(new_col)) + new_col
            for i in range(height):
                board[i][j] = new_col[i]
    elif direction == 0:  
        for j in range(width):
            new_col = [board[i][j] for i in range(height) if board[i][j] is not None]
            new_col = new_col + [None] * (height - len(new_col))
            for i in range(height):
                board[i][j] = new_col[i]
    elif direction == 3: 
        for i in range(height):
            new_row = [board[i][j] for j in range(width) if board[i][j] is not None]
            new_row = [None] * (width - len(new_row)) + new_row
            for j in range(width):
                board[i][j] = new_row[j]
    elif direction == 2:  
        for i in range(height):
            new_row = [board[i][j] for j in range(width) if board[i][j] is not None]
            new_row = new_row + [None] * (width - len(new_row))
            for j in range(width):
                board[i][j] = new_row[j]
    index = 0
    for i in range(height):
        for j in range(width):
            if board[i][j] is None:
                board[i][j] = lifted_elements[index]
                index += 1

    return board

In [3]:
import heapq
import time

# Hàm heuristic tính toán dựa trên số phần tử sai lệch (chi phí thấp hơn)
def heuristic(matrix, goal_matrix):
    return sum(sum(1 for a, b in zip(row1, row2) if a != b) for row1, row2 in zip(matrix, goal_matrix))

def find_die_and_move(data):
    start_time = time.time()
    start_matrix, goal_matrix = generate_board_matrix(data)
    height = len(start_matrix)
    width = len(start_matrix[0])

    # Khởi tạo hàng đợi ưu tiên (priority queue)
    open_set = []
    heapq.heappush(open_set, (0, start_matrix, []))
    
    # Lưu các trạng thái đã duyệt
    visited = set()
    visited.add(tuple(map(tuple, start_matrix)))

    # Duyệt trạng thái cho đến khi tìm thấy mục tiêu
    while open_set:
        # Lấy ra trạng thái có chi phí thấp nhất
        _, current_matrix, moves = heapq.heappop(open_set)

        # Kiểm tra nếu trạng thái hiện tại là mục tiêu
        if current_matrix == goal_matrix:
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"time: {elapsed_time:.2f} ")
            return moves

        # Lưu các vùng đã thay đổi để kiểm tra cục bộ thay vì kiểm tra toàn bộ ma trận
        changed_regions = set()

        for matrix_id in range(0, 26):
            die_pattern = generate_matrix(matrix_id, data)
            p_height = len(die_pattern)
            p_width = len(die_pattern[0])

            for i in range(height - p_height + 1):
                for j in range(width - p_width + 1):
                    for direction in range(4):
                        temp_matrix = [row[:] for row in current_matrix]
                        apply_die_cutting(temp_matrix, die_pattern, i, j, direction)
                        temp_tuple = tuple(map(tuple, temp_matrix))
                        
                        # Kiểm tra nếu trạng thái mới chưa được duyệt
                        if temp_tuple not in visited:
                            visited.add(temp_tuple)
                            cost = len(moves) + 1 + heuristic(temp_matrix, goal_matrix)
                            heapq.heappush(open_set, (cost, temp_matrix, moves + [(die_pattern, i, j, direction)]))
    
    return None


moves = find_die_and_move(data)
print(moves)


time: 0.12 
[([[0, 1, 0], [1, 0, 1], [1, 1, 0]], 1, 2, 2), ([[1, 1], [1, 1]], 3, 2, 2), ([[0, 1, 0], [1, 0, 1], [1, 1, 0]], 4, 5, 2), ([[1]], 6, 6, 2)]
